In [1]:
import coax
import gym
import haiku as hk
import jax
import jax.numpy as jnp
import optax
from coax.value_losses import mse
from RPPRegularizer import RPPRegularizer

import matplotlib.pyplot as plt

# the name of this script
name = 'a2c'

# the cart-pole MDP
# env = gym.make('CartPole-v0')
env = gym.make("rpp_gym:InclinedCartpole-v0")

In [2]:
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")

In [3]:
# env.spec.max_episode_steps = 200
# env.spec.reward_threshold = 195.0

In [4]:
from emlp import T, Scalar
from emlp.groups import SO, S, O, Trivial,Z
import emlp.nn.haiku as ehk
from emlp.reps import Rep
from emlp.nn import gated,gate_indices,uniform_rep
from math import prod
from representations import PseudoScalar
from mixed_emlp_haiku import MixedEMLP

## Trivial
# group=Trivial(2)
# rep_in = T(0)*prod(env.observation_space.shape)
# rep_out = T(0)*env.action_space.n#prod(env.action_space.shape)

## Reflection
group=Z(2)
rep_in = PseudoScalar()*prod(env.observation_space.shape)
rep_out = T(1)#*env.action_space.n#prod(env.action_space.shape)

nn_pi = ehk.EMLP(rep_in,rep_out,group,ch=100,num_layers=2)
nn_v = ehk.EMLP(rep_in,T(0),group,ch=100,num_layers=2)

# nn_pi = ehk.MLP(rep_in,rep_out(group),group,ch=100,num_layers=2)
# nn_v = ehk.MLP(rep_in,T(0),group,ch=100,num_layers=2)

# nn_pi = MixedEMLP(rep_in,rep_out(group),group,ch=100,num_layers=2)
# nn_v = MixedEMLP(rep_in,T(0),group,ch=100,num_layers=2)


def func_pi(S, is_training):
    return {'logits': nn_pi(S)}


def func_v(S, is_training):
    return nn_v(S).reshape(-1)



# def func_pi(S, is_training):
#     logits = hk.Sequential((
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(16), jax.nn.relu,
#         hk.Linear(env.action_space.n, w_init=jnp.zeros)
#     ))
#     return {'logits': logits(S)}



# def func_v(S, is_training):
#     value = hk.Sequential((
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(32), jax.nn.relu,
#         hk.Linear(1, w_init=jnp.zeros), jnp.ravel
#     ))
#     return value(S)


[a2c|root|INFO] Initing EMLP (Haiku)
[a2c|root|INFO] Linear W components:400 rep:96P+48P⊗V+20P⊗V²+8P⊗V³+4P⊗V⁴
[a2c|root|INFO] P cache miss
[a2c|root|INFO] Solving basis for P, for G=Z(2)
[a2c|root|INFO] P⊗V cache miss
[a2c|root|INFO] Solving basis for P⊗V, for G=Z(2)
[a2c|root|INFO] P⊗V² cache miss
[a2c|root|INFO] Solving basis for P⊗V², for G=Z(2)
[a2c|root|INFO] P⊗V³ cache miss
[a2c|root|INFO] Solving basis for P⊗V³, for G=Z(2)
[a2c|root|INFO] P⊗V⁴ cache miss
[a2c|root|INFO] Solving basis for P⊗V⁴, for G=Z(2)
[a2c|root|INFO] V cache miss
[a2c|root|INFO] Solving basis for V, for G=Z(2)
[a2c|root|INFO] V² cache miss
[a2c|root|INFO] Solving basis for V², for G=Z(2)
[a2c|root|INFO] V³ cache miss
[a2c|root|INFO] Solving basis for V³, for G=Z(2)
[a2c|root|INFO] V⁴ cache miss
[a2c|root|INFO] Solving basis for V⁴, for G=Z(2)
[a2c|root|INFO] Linear W components:10000 rep:576V⁰+576V+384V²+216V³+121V⁴+44V⁵+14V⁶+4V⁷+V⁸
[a2c|root|INFO] V⁵ cache miss
[a2c|root|INFO] Solving basis for V⁵, for G=Z(2

In [5]:
# these optimizers collect batches of grads before applying updates
optimizer_v = optax.chain(optax.apply_every(k=32), optax.adam(0.002))
optimizer_pi = optax.chain(optax.apply_every(k=32), optax.adam(0.001))


# value function and its derived policy
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)

In [6]:
# experience tracer
pi_regularizer = RPPRegularizer(pi, basic_wd=1e-4, equiv_wd=1e-4)
tracer = coax.reward_tracing.NStep(n=1, gamma=0.9)

# updaters
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optimizer_pi,
                                              regularizer=pi_regularizer)
simple_td = coax.td_learning.SimpleTD(v, loss_function=mse, optimizer=optimizer_v)

epoch_rewards = []

# train
for ep in range(1000):
    s = env.reset()
    er = 0
    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)
        
        if done and (t == env.spec.max_episode_steps - 1):
            r = 1 / (1 - tracer.gamma)
        er+=r
        tracer.add(s, a, r, done)
        while tracer:
            transition_batch = tracer.pop()
            metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
            metrics_pi = vanilla_pg.update(transition_batch, td_error)
            env.record_metrics(metrics_v)
            env.record_metrics(metrics_pi)

        if done:
            break

        s = s_next
    
    print("Epoch reward",er)
    epoch_rewards.append(er)
    # early stopping
    if env.avg_G > env.spec.reward_threshold:
        break


# run env one more time to render
#coax.utils.generate_gif(env, policy=pi, filepath=f"./data/{name}.gif", duration=25)

[a2c|TrainMonitor|INFO] ep: 1,	T: 12,	G: 11,	avg_r: 1,	avg_G: 11,	t: 11,	dt: 781.796ms,	SimpleTD/loss: 0.349,	VanillaPG/loss: 1


Epoch reward 11.0


[a2c|TrainMonitor|INFO] ep: 2,	T: 71,	G: 58,	avg_r: 1,	avg_G: 34.5,	t: 58,	dt: 27.695ms,	SimpleTD/loss: 0.371,	VanillaPG/loss: 0.974


Epoch reward 58.0


[a2c|TrainMonitor|INFO] ep: 3,	T: 95,	G: 23,	avg_r: 1,	avg_G: 30.7,	t: 23,	dt: 29.031ms,	SimpleTD/loss: 0.755,	VanillaPG/loss: 0.838


...

Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 148,	T: 14,335,	G: 200,	avg_r: 1,	avg_G: 194,	t: 200,	dt: 31.016ms,	SimpleTD/loss: 0.00026,	VanillaPG/loss: 0.515


Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 149,	T: 14,536,	G: 200,	avg_r: 1,	avg_G: 195,	t: 200,	dt: 28.119ms,	SimpleTD/loss: 0.000225,	VanillaPG/loss: 0.516


Epoch reward 209.0
Epoch reward 209.0


In [7]:
metrics_pi

{'RPPRegularizer/basic_l2': DeviceArray(0., dtype=float32),
 'RPPRegularizer/basic_wd': DeviceArray(1.e-04, dtype=float32),
 'RPPRegularizer/equiv_l2': DeviceArray(5165.331, dtype=float32),
 'RPPRegularizer/equiv_wd': DeviceArray(1.e-04, dtype=float32),
 'VanillaPG/grads_max': DeviceArray(0.08917519, dtype=float32),
 'VanillaPG/grads_norm': DeviceArray(0.48706943, dtype=float32),
 'VanillaPG/kl_div_old': DeviceArray(0.23434481, dtype=float32),
 'VanillaPG/loss': DeviceArray(0.48180765, dtype=float32),
 'VanillaPG/loss_bare': DeviceArray(-0.03472544, dtype=float32)}

In [8]:
equiv_l2 = 0.0
basic_l2 = 0.0
for k1, v1 in pi.params.items():
    if "bi" not in k1:
        for k2, v2 in v1.items():
    #         if "bi" not in k2:
    #             print(k2)
            if k2.endswith("_basic"):
                basic_l2 += (v2 ** 2).sum()
            elif k2.endswith("w"):
                equiv_l2 += (v2 ** 2).sum()
print("Eq l2 = ", equiv_l2)
print("Basic l2 = ", basic_l2)

Eq l2 =  338.9434
Basic l2 =  0.0
